# DOWNLOADING DATA

In [1]:
!kaggle datasets download -d devicharith/language-translation-englishfrench

Dataset URL: https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench
License(s): CC0-1.0
 57%|█████████████████████▋                | 2.00M/3.51M [00:00<00:00, 2.93MB/s]
100%|██████████████████████████████████████| 3.51M/3.51M [00:00<00:00, 4.04MB/s]


In [2]:
!unzip language-translation-englishfrench.zip -d language-translation-englishfrench

Archive:  language-translation-englishfrench.zip
  inflating: language-translation-englishfrench/eng_-french.csv  


# IMPORTS

In [3]:
import pandas as pd

# DATA EXPROLATION

In [25]:
english_french_data = pd.read_csv('language-translation-englishfrench/eng_-french.csv', encoding='utf-8')
print(f'Number of sentences: {len(english_french_data)}')
english_sentence_words = english_french_data['English words/sentences'].apply(lambda x: len(str(x).split()))
french_avg_sentence_words = english_french_data['French words/sentences'].apply(lambda x: len(str(x).split()))
print(f'Min words in english sentences: {english_sentence_words.min()}')
print(f'Min words in french sentences: {french_avg_sentence_words.min()}')
print(f'Max words in english sentences: {english_sentence_words.max()}')
print(f'Max words in french sentences: {french_avg_sentence_words.max()}')
print(f'Average words in english sentences: {english_sentence_words.mean()}')
print(f'Average words in french sentences: {french_avg_sentence_words.mean()}')
english_french_data.head()

Number of sentences: 175621
Min words in english sentences: 1
Min words in french sentences: 1
Max words in english sentences: 44
Max words in french sentences: 55
Average words in english sentences: 6.161552433934438
Average words in french sentences: 6.706669475746067


,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [19]:
english_french_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   English words/sentences  175621 non-null  object
 1   French words/sentences   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB
